#YOLOv4 Drone detection

In [ ]:
import os
import shutil
import time
from google.colab import drive
from glob import glob
import numpy as np

In [ ]:
!nvidia-smi

In [ ]:
drive.mount("/content/drive")

##Cloning repositories

In [ ]:
!git clone https://github.com/tjuric03/DroneDetection
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
%cd darknet

##Installing dependencies

In [ ]:
!apt install libopencv-dev python-opencv ffmpeg

##Building darknet

In [ ]:
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!sed -i 's/GPU=0/GPU=1/g' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/g' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/g' Makefile

!make

##Downloading weights

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.weights

#Training YOLOv4-tiny model

## Generate .cfg file

In [ ]:
CFG_MODEL_NAME = "yolov4-tiny-4class.cfg"

TINY_WEIGHTS_PATH = "yolov4-tiny.weights"
TINY_WEIGHTS_29_PATH = "yolov4-tiny.conv.29"
BASE_CONFIG_PATH = "cfg/yolov4-tiny-custom.cfg"
CUSTOM_CONFIG_PATH = f"cfg/{CFG_MODEL_NAME}"

In [ ]:
!./darknet partial {BASE_CONFIG_PATH} {TINY_WEIGHTS_PATH} {TINY_WEIGHTS_29_PATH} 29

In [ ]:
!cp {BASE_CONFIG_PATH} {CUSTOM_CONFIG_PATH}

In [ ]:
NUMBER_OF_CLASSES = 4
BATCH_SIZE = 64
SUBDIVISIONS = 16
MAX_BATCHES = max(6000, NUMBER_OF_CLASSES * 2000)
STEP1 =  int(0.8 * MAX_BATCHES)
STEP2 = int(0.9 * MAX_BATCHES)
WIDTH = 416
HEIGHT = 416
RANDOM = 1


!sed -i 's/^classes=.*/classes={NUMBER_OF_CLASSES}/g' {CUSTOM_CONFIG_PATH}
!sed -i 's/^batch=.*/batch={BATCH_SIZE}/g' {CUSTOM_CONFIG_PATH}
!sed -i 's/^subdivisions=.*/subdivisions={SUBDIVISIONS}/g' {CUSTOM_CONFIG_PATH}
!sed -i 's/^max_batches.*/max_batches={MAX_BATCHES}/g' {CUSTOM_CONFIG_PATH}
!sed -i 's/^steps=.*/steps={STEP1},{STEP2}/g' {CUSTOM_CONFIG_PATH}
!sed -i 's/^width=.*/width={WIDTH}/g' {CUSTOM_CONFIG_PATH}
!sed -i 's/^height=.*/height={HEIGHT}/g' {CUSTOM_CONFIG_PATH}
!sed -i 's/^random=.*/random={RANDOM}/g' {CUSTOM_CONFIG_PATH}

#NOTE: MUST CHANGE filters=(classes+5)x3 in each convolutional layer before yolo MANUALLY

## Load config

In [ ]:
CONFIG_NAME = "yolov4-tiny-4class.cfg"

CONFIGS_PATH = "/content/drive/MyDrive/training/Configs/"
CONFIG_LOAD_PATH = CONFIGS_PATH+CONFIG_NAME

if(os.path.exists(CONFIG_LOAD_PATH)):
  print(f"Replacing config at path {CUSTOM_CONFIG_PATH}")
  shutil.copyfile(CONFIG_LOAD_PATH,CUSTOM_CONFIG_PATH)
else:
  print(f"No config found at path {CONFIG_LOAD_PATH}")

In [ ]:
DATA_DIRECTORY_PATH = "build/darknet/x64/data"

with open(f"{DATA_DIRECTORY_PATH}/obj.names", "w") as fp:
    fp.write("""drone
bird
plane
heli
""")

BACKUP_TRAINING_PATH = "/content/drive/MyDrive/training/yolov4-tiny-4class_"+time.strftime("%Y%m%d-%H%M%S")

os.mkdir(BACKUP_TRAINING_PATH)

with open(f"{DATA_DIRECTORY_PATH}/obj.data", "w") as fp:
    fp.write(f"""classes = {NUMBER_OF_CLASSES}
train  = build/darknet/x64/data/train.txt
valid  = build/darknet/x64/data/test.txt
names = build/darknet/x64/data/obj.names
backup = {BACKUP_TRAINING_PATH}""")



In [ ]:
!cp "{DATA_DIRECTORY_PATH}/obj.names" {BACKUP_TRAINING_PATH}
!cp "{DATA_DIRECTORY_PATH}/obj.data" {BACKUP_TRAINING_PATH}
!cp "{CUSTOM_CONFIG_PATH}" {BACKUP_TRAINING_PATH}

In [ ]:
IMAGES_SOURCE_PATH = "/content/DroneDetection/Data"

IMAGES_DESTINATION_PATH = "build/darknet/x64/data/obj/"

!cp -r {IMAGES_SOURCE_PATH} {IMAGES_DESTINATION_PATH}

### Stratified sampling

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd

images = pd.DataFrame(glob(f"{IMAGES_DESTINATION_PATH}/*/*.JPEG"),columns=["image_path"])

images["type"] = images.apply(lambda row: row[0].split("/")[-2],axis=1)

train, test = train_test_split(images,test_size=0.1,stratify=images["type"])

In [ ]:
train["image_path"].to_csv(f"{DATA_DIRECTORY_PATH}/train.txt",index=False,header=False)
test["image_path"].to_csv(f"{DATA_DIRECTORY_PATH}/test.txt",index=False,header=False)

In [ ]:
#images = glob(f"{IMAGES_DESTINATION_PATH}/*/*.JPEG")

#with open(f"{DATA_DIRECTORY_PATH}/train.txt", "w") as fp:
#    for image in images:
#      fp.write(f"{image}\n")

In [ ]:
#!./darknet detector train "{DATA_DIRECTORY_PATH}/obj.data" {DRONE_CONFIG_PATH} {TINY_WEIGHTS_29_PATH} -dont_show -map
!./darknet detector train "{DATA_DIRECTORY_PATH}/obj.data" {CUSTOM_CONFIG_PATH} {TINY_WEIGHTS_29_PATH} -dont_show -map

## Testing trained YOLOv4-tiny model

In [ ]:
!./darknet detect cfg/yolov4.cfg yolov4.weights -thresh 0.25 data/spg3.jpg

In [ ]:
from google.colab.patches import cv2_imshow
#!curl -o logo.png https://colab.research.google.com/img/colab_favicon_256px.png
import cv2
img = cv2.imread('../DroneDetection/Data/Drone/drone_0.JPEG', cv2.IMREAD_UNCHANGED)
cv2_imshow(img)

In [ ]:
chart.png